In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [3]:
# 加载ImageCollection
col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI')
# 掩膜
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'Africa'))
# 定义gif生成区域
region = ee.Geometry.Polygon(
  [[[-18.698368046353494, 38.1446395611524],
    [-18.698368046353494, -36.16300755581617],
    [52.229366328646506, -36.16300755581617],
    [52.229366328646506, 38.1446395611524]]],
  None, False
)
Map.addLayer(region,{},'region')
Map.centerObject(region,6)

In [4]:
def set_doy(img):
    doy = ee.Date(img.get('system:time_start')).getRelative('day', 'year')
    return img.set('doy', doy)
col = col.map(set_doy)

In [5]:
# 二维ImageCollection的制作
distinctDOY = col.filterDate('2013-01-01', '2014-01-01')
filter_ = ee.Filter.equals(**{'leftField': 'doy', 'rightField': 'doy'})
join = ee.Join.saveAll('doy_matches')
joinCol = ee.ImageCollection(join.apply(distinctDOY, col, filter_))

In [6]:

def apply_median_redution(img):
    doyCol = ee.ImageCollection.fromImages(img.get('doy_matches'))
    return doyCol.reduce(ee.Reducer.median())
comp = joinCol.map(apply_median_redution)  

In [7]:
# 可视化参数
visParams = {
  'min': 0.0,
  'max': 9000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
rgbVis = comp.map(lambda x:x.visualize(**visParams).clip(mask))
Map.addLayer(rgbVis)

In [8]:
gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:3857',
  'framesPerSecond': 10
}

geemap.download_ee_video(rgbVis,gifParams,'linodebug.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\DELL\gee\linodebug.gif


In [ ]:
# 可视化参数
visParams = {
  'dimensions': 768,
  'region': region,
  'framesPerSecond': 7,
  'crs': 'EPSG:3857',
  'min': 0.0,
  'max': 9000.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}
print(comp.getVideoThumbURL(visParams))